In [233]:
import time
import os
import sys
from math import radians, cos, sin, asin, sqrt
import folium
import webbrowser
import pandas as pd
import csv

In [234]:
time_format = '%Y-%m-%d %H:%M:%S'

# structure of point
class Point:
    def __init__(self, latitude, longitude, dateTime, arriveTime, leaveTime):
        self.latitude = latitude
        self.longitude = longitude
        self.dateTime = dateTime
        self.arriveTime = arriveTime
        self.leaveTime = leaveTime

In [235]:
# calculate distance between two points from their coordinate
def getDistanceOfPoints(pi, pj):
    lat1, lon1, lat2, lon2 = list(map(radians, [float(pi.latitude), float(pi.longitude),
                                                float(pj.latitude), float(pj.longitude)]))
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    m = 6371000 * c
    return m

# calculate time interval between two points
def getTimeIntervalOfPoints(pi, pj):
    t_i = time.mktime(time.strptime(pi.dateTime, time_format))
    t_j = time.mktime(time.strptime(pj.dateTime, time_format))
    return t_j - t_i

# compute mean coordinates of a group of points
def computMeanCoord(gpsPoints):
    lat = 0.0
    lon = 0.0
    for point in gpsPoints:
        lat += float(point.latitude)
        lon += float(point.longitude)
    return (lat/len(gpsPoints), lon/len(gpsPoints))

In [231]:
# extract stay points from a GPS log file
# input:
#        file: the name of a GPS log file
#        distThres: distance threshold
#        timeThres: time span threshold
# default values of distThres and timeThres are 200 m and 30 min respectively, according to [1]
def stayPointExtraction(points, distThres=200, timeThres=30 * 60):
    stayPointList = []
    stayPointCenterList = []
    pointNum = len(points)
    i = 0

    while i < pointNum:
        j = i + 1
        while j < pointNum:
            if getDistanceOfPoints(points[i], points[j]) > distThres:
                # points[j] has gone out of bound thus it should not be counted in the stay points.
                if getTimeIntervalOfPoints(points[i], points[j-1]) > timeThres:
                    latitude, longitude = computMeanCoord(points[i:j])
                    arriveTime = time.mktime(time.strptime(points[i].dateTime, time_format))
                    leaveTime = time.mktime(time.strptime(points[j-1].dateTime, time_format))
                    dateTime = time.strftime(time_format, time.localtime(arriveTime)), time.strftime(time_format, time.localtime(leaveTime))
                    stayPointCenterList.append(Point(latitude, longitude, dateTime, arriveTime, leaveTime))
                    stayPointList.extend(points[i:j])
                break
            j += 1
        i = j

    # while i < pointNum:
    #     j = i + 1
    #     while j < pointNum:
    #         s = getDistanceOfPoints(points[i], points[j])
    #         t = getTimeIntervalOfPoints(points[i], points[j-1])
    #         if t > 0:
    #             velocity = s/t
    #             velThres = 2.8
    #             if velocity < velThres:
    #                 latitude, longitude = computMeanCoord(points[i:j])
    #                 arriveTime = time.mktime(time.strptime(points[i].dateTime, time_format))
    #                 leaveTime = time.mktime(time.strptime(points[j-1].dateTime, time_format))
    #                 dateTime = time.strftime(time_format, time.localtime(arriveTime)), time.strftime(time_format, time.localtime(leaveTime))
    #                 stayPointCenterList.append(Point(latitude, longitude, dateTime, arriveTime, leaveTime))
    #                 stayPointList.extend(points[i:j])
    #             break
    #         j += 1
    #     i = j
    return stayPointCenterList, stayPointList

# parse lines into points
def parseGeoTxt(df):
    points = []
    for index, row in df.iterrows():
        latitude = float(row[1])
        longitude = float(row[2])
        dateTime = row[3]
        points.append(Point(latitude, longitude, dateTime, 0, 0))
    return points

# add points into mapDots (type: folium.map.FeatureGroup())
def addPoints(mapDots, points, color):
    for p in points:
        mapDots.add_child(folium.CircleMarker(
            [p.latitude, p.longitude], 
            radius=4,
            tooltip=p.dateTime,
            color=color,
            ))

In [232]:
m = folium.Map(location=[-7.794847,110.36560821533205])
tooltip = "hello"
mapDots = folium.map.FeatureGroup()

path = "../../../DataGPS_Malioboro/filter2_maljan_des.csv"
filename = path.replace('filter2', 'filter3_basic')
print("processing:" ,  path)

# Baca file CSV menjadi DataFrame
df = pd.read_csv(path)

# Kelompokkan data berdasarkan 'maid'
grouped_data = df.groupby(['maid', 'tanggal'])
i = 0
stayPoint_df = pd.DataFrame(columns=['maid', 'latitude', 'longitude', 'arriveTime', 'leaveTime'])

# Lakukan operasi apa pun pada setiap kelompok data
for (maid, tanggal), data in grouped_data:
    data = data.sort_values(by='datetime_wib')
    points = parseGeoTxt(data)
    stayPointCenter, stayPoint = stayPointExtraction(points)
    
    if len(stayPointCenter) > 0:
        for sp in stayPointCenter:
            stayPoint_df = stayPoint_df._append({'maid':maid, 'latitude': sp.latitude, 'longitude': sp.longitude, 'arriveTime': time.strftime(time_format, time.localtime(sp.arriveTime)), 'leaveTime': time.strftime(time_format, time.localtime(sp.leaveTime))}, ignore_index=True)   
    
print(len(stayPoint_df['maid'].unique()), "maid has stay point")
stayPoint_df.to_csv(filename, index=False)

processing: ../../../DataGPS_Malioboro/filter2_maljan_des.csv


33 maid has stay point


In [227]:
stayPoint_df

,maid,latitude,longitude,arriveTime,leaveTime
0,00012afc-6daf-461f-96a8-181e5af69db9,-7.800291,110.364998,2021-12-01 17:54:11,2021-12-01 17:54:12
1,00bdab07-a389-41d8-bb5a-743349b86e7f,-7.792231,110.365899,2021-12-17 16:49:48,2021-12-17 16:50:15
2,00f39daa-44b2-45c6-ae04-dd7b8a020452,-7.792019,110.365875,2021-12-26 10:34:27,2021-12-26 10:47:34
3,00f39daa-44b2-45c6-ae04-dd7b8a020452,-7.792039,110.365875,2021-12-26 12:07:27,2021-12-26 16:37:31
4,01e2cf9a-ed42-4131-b506-5fa89496afe7,-7.798815,110.365315,2021-12-11 12:23:17,2021-12-11 12:23:27
...,...,...,...,...,...
3857,fff98271-0aa0-49dd-9552-0566aac0e6c5,-7.796339,110.365555,2021-12-01 01:16:15,2021-12-01 01:17:52
3858,fff98271-0aa0-49dd-9552-0566aac0e6c5,-7.796345,110.365557,2021-12-01 01:19:37,2021-12-01 01:28:47
3859,fff98271-0aa0-49dd-9552-0566aac0e6c5,-7.796339,110.365555,2021-12-01 01:29:26,2021-12-01 01:30:41
3860,fff98271-0aa0-49dd-9552-0566aac0e6c5,-7.796339,110.365555,2021-12-01 01:31:20,2021-12-01 01:32:48


In [51]:
m = folium.Map(location=[-7.794847,110.36560821533205])
tooltip = "hello"
mapDots = folium.map.FeatureGroup()

gpsfile = "../../../DataGPS_Malioboro/filter2_malioboro_des.csv"
print("processing:" ,  gpsfile)

log = open(gpsfile, 'r')
lines = log.readlines()[1:]
points = parseGeoTxt(lines)

stayPointCenter, stayPoint = stayPointExtraction(points)
addPoints(mapDots, points, 'yellow')

if len(stayPointCenter) > 0:
    # add pionts to a group to be shown on map
    addPoints(mapDots, stayPoint, 'blue')
    addPoints(mapDots, stayPointCenter, 'red')

    spfile = gpsfile.replace('filter2', 'filter3_basic')
    if not os.path.exists(os.path.dirname(spfile)):
        os.makedirs(os.path.dirname(spfile))
    spfile_handle = open(spfile, 'w+')
    
    print('latitude, longitude, arriving time, leaving time', file=spfile_handle)
    for sp in stayPointCenter:
        print(sp.latitude, sp.longitude, time.strftime(time_format, time.localtime(sp.arriveTime)), time.strftime(time_format, time.localtime(sp.leaveTime)), file=spfile_handle)
    spfile_handle.close()

    print("writen into:" ,  spfile) 

else:
    print(gpsfile , "has no stay point")

print("Contain" , len(stayPointCenter) , "stay points")

processing: ../../../DataGPS_Malioboro/filter2_malioboro_des.csv
writen into: ../../../DataGPS_Malioboro/filter3_basic_malioboro_des.csv
Contain 681 stay points


In [45]:
for sp in stayPointCenter:
    print(sp.latitude, sp.longitude)

-7.793051999999997 110.36655569751882
-7.793382999999997 110.36614471293134
-7.793154140740739 110.36652407845956
-7.793719588235296 110.3659273467084
-7.7930607499999995 110.36602761427136
-7.791451281944455 110.36643633141414
-7.789829999999998 110.36663985468753
-7.793836866666666 110.36663818359376
-7.791278016666667 110.36687177907311
-7.793844825000001 110.36651494862878
-7.794685274999997 110.36663852416993
-7.794158618518517 110.36648980215568
-7.791658230769231 110.36633000460111
-7.793773696296296 110.36614861093028
-7.794512767857141 110.36647643275674
-7.79146555 110.36669409179689
-7.795204666666668 110.36590757202147
-7.794553999999999 110.36663999999999
-7.790760333333334 110.36617423095703
-7.793802763750003 110.36627833095704
-7.793622319230769 110.36610412494367
-7.793086250000004 110.36611748316086
-7.792282285714286 110.36608123779297
-7.790338799999999 110.36628495996095
-7.795266714285714 110.36567423688616
-7.791539712499999 110.36630236206055
-7.794812 110.36628